In [17]:
from pycompiler import Grammar

G = Grammar()

Program = G.NonTerminal('Program', True)

# ----------------------------------------------------------------------------------------------------------------
#                                       Definiendo los No Terminales
Type_function_list = G.NonTerminal('Type_function_list')
Expr_block, Expr_list, Expr_item_list, Expr, Simple_expr = G.NonTerminals('Expr_block Expr_list Expr_item_list Expr Simple_expr')
Or_expr, And_expr, Aritm_comp = G.NonTerminals('Or_expr And_expr Aritm_comp') 
Concat = G.NonTerminal('Concat')
Arithmetic, Term, Pow, Factor = G.NonTerminals('Arithmetic Term Pow Factor')
Conditional, Cond_other_case = G.NonTerminals('Conditional Cond_other_case')
Let_expr, Assignment = G.NonTerminals('Let_expr Assignment')
Destr_assig = G.NonTerminal('Destr_assig')
Atom, Call_func, Member = G.NonTerminals('Atom Call_func Member')
Variable, Params, Method_dec, Inline_form, Full_form, Func, Arguments = G.NonTerminals('Variable Params Method_dec Inline_form Full_form Func Arguments')
While_loop, For_loop = G.NonTerminals('While_loop For_loop')
Cast_type = G.NonTerminal('Cast_type')
Check_type = G.NonTerminal('Check_type')
Sign = G.NonTerminal('Sign')
Type_block, Type_member_list, Type_member_item, Type, Type_dec, Type_inst = G.NonTerminals('Type_block Type_member_list Type_member_item Type Type_dec Type_inst')
Protocol_block, Method_dec_list, Protocol = G.NonTerminals('Protocol_block Method_dec_list Protocol')
Vector, Vector_exp, Vector_imp, Vector_item_list, Index_object = G.NonTerminals('Vector Vector_exp Vector_imp Vector_item_list Index_object')

# ----------------------------------------------------------------------------------------------------------------
#                                       Definiendo los Terminales
o_curly, c_curly, semicolon, o_par, c_par, comma, colon, dot, destr_op, arrow, o_square_bracket, c_square_bracket, such_that_op = G.Terminals('{ } ; ( ) , : . := => [ ] ||')
or_op, and_op, equal, not_equal, gth, geth, lth, leth, not_op, assign_op = G.Terminals('| & == != > >= < <= ! =')
concat_op, double_concat_op = G.Terminals('@ @@')
plus_op, minus_op, mult_op, div_op, pow_op, double_star_op, mod_op = G.Terminals('+ - * / ^ ** %')
false, true, id_, num, let, in_, function_, string_ = G.Terminals('false true id num let in function string')
if_, elif_, else_  = G.Terminals('if elif else')
as_, is_ = G.Terminals('as is')
while_, for_ = G.Terminals('while for')
type_, inherits, new = G.Terminals('type inherits new')
protocol_, extends = G.Terminals('protocol extends')

# ----------------------------------------------------------------------------------------------------------------
#                                               Producciones
Program %= Type_function_list + Expr_item_list 

Type_function_list %= Func + Type_function_list
Type_function_list %= Type + Type_function_list
Type_function_list %= Protocol + Type_function_list
Type_function_list %= G.Epsilon

# Un bloque de expresiones es una lista de expresiones entre { }
Expr_block %= o_curly + Expr_list + c_curly

# Una lista de expresiones es un conjunto de expresiones de item list
Expr_list %= Expr_item_list + Expr_list 
Expr_list %= Expr_item_list
# Expr_list %= Expr_block + Expr_list
# Expr_list %= G.Epsilon

# Un elemento de una lista de expresiones puede ser una expresion seguida de un ';' o un bloque de expresiones
Expr_item_list %= Expr + semicolon 
# Expr_item_list %= Expr_block

# Una expresion puede ser un bloque de expresiones o una expresion simple
# Expr %= Expr_block
Expr %= Simple_expr

# Una expresion simple puede ser un condicional, una expresion let o un destructive assignment
Simple_expr %= Conditional
Simple_expr %= Let_expr
Simple_expr %= While_loop
Simple_expr %= For_loop
Simple_expr %= Destr_assig

Destr_assig %= Variable + destr_op + Expr
Destr_assig %= Or_expr 

Or_expr %= Or_expr + or_op + And_expr
Or_expr %= And_expr

And_expr %= And_expr + and_op + Check_type
And_expr %= Check_type

Check_type %= Check_type + is_ + Aritm_comp
Check_type %= Aritm_comp

Aritm_comp %= Aritm_comp + equal + Concat
Aritm_comp %= Aritm_comp + not_equal + Concat
Aritm_comp %= Aritm_comp + gth + Concat
Aritm_comp %= Aritm_comp + geth + Concat
Aritm_comp %= Aritm_comp + lth + Concat
Aritm_comp %= Aritm_comp + leth + Concat
Aritm_comp %= Concat

Concat %= Concat + concat_op + Arithmetic
Concat %= Concat + double_concat_op + Arithmetic
Concat %= Arithmetic 

Arithmetic %= Arithmetic + plus_op + Term 
Arithmetic %= Arithmetic + minus_op + Term 
Arithmetic %= Term

Term %= Term + mult_op + Pow
Term %= Term + div_op + Pow 
Term %= Term + mod_op + Pow
Term %= Pow

Pow %= Sign + pow_op + Pow
Pow %= Sign + double_star_op + Pow 
Pow %= Sign

Sign %= plus_op + Factor
Sign %= minus_op + Factor
Sign %= Factor

Factor %= not_op + Atom
Factor %= Atom

Atom %= o_par + Expr + c_par
Atom %= Expr_block
Atom %= false
Atom %= true
Atom %= Call_func
Atom %= Type_inst
Atom %= Vector
Atom %= Index_object
Atom %= id_
Atom %= Member
Atom %= num
Atom %= string_
Atom %= Cast_type

Conditional %= if_ + o_par + Expr + c_par +  Expr + Cond_other_case
Cond_other_case %= elif_ + o_par + Expr + c_par + Expr + Cond_other_case 
Cond_other_case %= else_ + Expr

Let_expr %= let + Assignment + in_ + Expr

Assignment %= Variable + assign_op + Expr + comma + Assignment
Assignment %= Variable + assign_op + Expr

Variable %= id_ + colon + id_
Variable %= id_

# Un parametro es una variable que recibe una funcion cuando es declarada
Params %= Variable + comma + Params
Params %= Variable 

Method_dec %= id_ + o_par + Params + c_par + colon + id_
Method_dec %= id_ + o_par + Params + c_par
Method_dec %= id_ + o_par + c_par + colon + id_
Method_dec %= id_ + o_par + c_par

Inline_form %= Method_dec + arrow + Simple_expr + semicolon

Full_form %= Method_dec + Expr_block + semicolon
Full_form %= Method_dec + Expr_block 

Func %= function_ + Inline_form
Func %= function_ + Full_form

# Un argumento es un valor que le pasamos a una funcion cuando se va a llamar
Arguments %= Expr + comma + Arguments
Arguments %= Expr

Call_func %= id_ + o_par + Arguments + c_par
Call_func %= id_ + o_par + c_par

Member %= Atom + dot + Call_func 
Member %= Atom + dot + id_

While_loop %= while_ + o_par + Expr + c_par + Expr

For_loop %= for_ + o_par + id_ + in_ + Expr + c_par + Expr

Type_block %= o_curly + Type_member_list + c_curly

Type_member_list %= Type_member_item + Type_member_list
Type_member_list %= G.Epsilon

Type_member_item %= Inline_form + semicolon
Type_member_item %= Full_form
Type_member_item %= Variable + assign_op + Expr + semicolon

Type %= type_ + Type_dec + Type_block
Type %= type_ + Type_dec + inherits + Type_dec + Type_block

Type_dec %= id_ + o_par + Params + c_par
Type_dec %= id_

Type_inst %= new + Call_func

Protocol_block %= o_curly + Method_dec_list + c_curly

Method_dec_list %= Method_dec + semicolon + Method_dec_list
Method_dec_list %= G.Epsilon

Protocol %= protocol_ + id_ + extends + id_ + Protocol_block
Protocol %= protocol_ + id_ + Protocol_block

Vector %= Vector_exp
Vector %= Vector_imp

Vector_exp %= o_square_bracket + Vector_item_list + c_square_bracket
Vector_exp %= o_square_bracket + c_square_bracket

Vector_item_list %= Expr + comma + Vector_item_list
Vector_item_list %= Expr

Vector_imp %= o_square_bracket + Expr + such_that_op + id_ + in_ + Expr + c_square_bracket

Index_object %= Atom + o_square_bracket + Expr + c_square_bracket

Cast_type %= Atom + as_ + id_

from parsing import LR1Parser, SLR1Parser

parse = SLR1Parser(G, verbose=True)
print('Exito')

Exito


In [8]:
print(G)

Non-Terminals:
	Program, Type_function_list, Expr_block, Expr_list, Expr_item_list, Expr, Simple_expr, Or_expr, And_expr, Aritm_comp, Concat, Arithmetic, Term, Pow, Factor, Conditional, Cond_other_case, Let_expr, Assignment, Destr_assig, Atom, Call_func, Member, Variable, Params, Method_dec, Inline_form, Full_form, Func, Arguments, While_loop, For_loop, Cast_type, Check_type, Sign, Type_block, Type_member_list, Type_member_item, Type, Type_dec, Type_inst, Protocol_block, Method_dec_list, Protocol, Vector, Vector_exp, Vector_imp, Vector_item_list, Index_object
Terminals:
	{, }, ;, (, ), ,, :, ., :=, =>, [, ], ||, |, &, ==, !=, >, >=, <, <=, !, =, @, @@, +, -, *, /, ^, **, %, false, true, id, num, let, in, function, string, if, elif, else, as, is, while, for, type, inherits, new, protocol, extends
Productions:
	[Program -> Type_function_list Expr_item_list, Type_function_list -> Func Type_function_list, Type_function_list -> Type Type_function_list, Type_function_list -> Protocol Type_fu